In [ ]:
import pandas as pd
import numpy as np

In [ ]:
imdb_data=pd.read_csv('/IMDB Dataset.csv')
imdb_data=imdb_data[:10000]
imdb_data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
d={"sentiment":{"positive":1,"negative":0}}
imdb_data=imdb_data.replace(d)
print(imdb_data["sentiment"])

0       1
1       1
2       1
3       0
4       1
       ..
9995    1
9996    0
9997    0
9998    0
9999    1
Name: sentiment, Length: 10000, dtype: int64


In [ ]:
imdb_data['sentiment'].value_counts()

1    5028
0    4972
Name: sentiment, dtype: int64

In [ ]:
import re
#Removing the html strips
def strip_html(text):
    return re.sub('(<br\s*/><br\s*/>)|(\-)|(\/)', '', text)
#Removing the square brackets
def remove_between_square_brackets(text):
    return re.sub('[.;:!\'?,\"()\[\]]', '', text)
#Removing the noisy text
def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    return text
#Apply function on review column
imdb_data['review']=imdb_data['review'].apply(denoise_text)

In [ ]:
def remove_special_characters(text, remove_digits=True):
    pattern=r'[^a-zA-z0-9\s]'
    text=re.sub(pattern,'',text)
    return text
#Apply function on review column
imdb_data['review']=imdb_data['review'].apply(remove_special_characters)

In [ ]:
import spacy
nlp=spacy.load('en', disable=['parser', 'ner'])

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
stop_words = set(stopwords.words('english'))
def remove(text):
  word_tokens = word_tokenize(text) 
  filtered_sentence = [w for w in word_tokens if not w in stop_words]   
  filtered_sentence = []  
  for w in word_tokens: 
    if w not in stop_words: 
        filtered_sentence.append(w) 
  filter=' '.join(filtered_sentence)
  return filter
imdb_data['review']=imdb_data['review'].apply(remove)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer()
nltk.download('wordnet')
def lemma(text):
  word_tokens = word_tokenize(text)
  lemmatized_output = ' '.join([lemmatizer.lemmatize(w) for w in word_tokens])
  return lemmatized_output
imdb_data['review']=imdb_data['review'].apply(lemma)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
from sklearn.model_selection import train_test_split
X=imdb_data['review']
Y=imdb_data['sentiment']
X1=imdb_data['review']
Y1=imdb_data['sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.40, random_state=42)
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, Y1, test_size=0.40, random_state=42)

In [ ]:
X_train=imdb_data.review[:6000]
X_test=imdb_data.review[6000:]
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
stop_words = set(stopwords.words('english'))
vectorizer1 = CountVectorizer(binary=False, ngram_range=(1, 3), stop_words=stop_words)
cv_train =vectorizer1.fit_transform(X_train)
cv_test=vectorizer1.transform(X_test)
vectorizer2 =TfidfVectorizer( ngram_range=(1, 3), stop_words=stop_words)
tf_train=vectorizer2.fit_transform(X_train)
tf_test=vectorizer2.transform(X_test)


In [ ]:
Y_train=imdb_data.sentiment[:6000]
Y_test=imdb_data.sentiment[6000:]

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier

In [ ]:
log = LogisticRegression(solver="lbfgs", random_state=42)
rnd = RandomForestClassifier(n_estimators=100, random_state=42)
nvb = GaussianNB()
voting = VotingClassifier(estimators=[('lr', log), ('rf', rnd),('nv',nvb)],voting='hard')

In [ ]:
from sklearn.metrics import accuracy_score
model1=log.fit(cv_train, Y_train)
y_pred = model1.predict(cv_test)
print("logistic regression", accuracy_score(Y_test, y_pred))

logistic regression 0.86825


In [ ]:
from sklearn.metrics import accuracy_score
model2=rnd.fit(cv_train, Y_train)
y_pred = model2.predict(cv_test)
print("random forest", accuracy_score(Y_test, y_pred))

random forest 0.82775


In [ ]:
#from sklearn.metrics import accuracy_score
#model3=nvb.fit(cv_train.toarray(), Y_train.toarray())
#y_pred = model3.predict(cv_test.toarray())
#print("naive bayes", accuracy_score(Y_test.toarray(), y_pred.toarray()))

In [ ]:
regularization = [0.01, 0.05, 0.25, 0.5, 1]
accuracy=[]
for c in regularization:
    lr = LinearSVC(C=c)
    lr.fit(cv_train, Y_train)
    accuracy.append(accuracy_score(Y_test, lr.predict(cv_test)))
    print("Accuracy for C=%s: %s" % (c, accuracy_score(Y_test, lr.predict(cv_test))))
# train final model with highest c
highestC = [regularization[i] for i in range(len(regularization)) if accuracy[i] == max(accuracy)][0]
final_model = LinearSVC(C = highestC)
final_model.fit(cv_train, Y_train)
y_pred = final_model.predict(cv_test)
print("Final Accuracy: %s" % accuracy_score(Y_test, y_pred))

Accuracy for C=0.01: 0.86925
Accuracy for C=0.05: 0.8685
Accuracy for C=0.25: 0.86875
Accuracy for C=0.5: 0.86825


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Accuracy for C=1: 0.8685
Final Accuracy: 0.86925


In [ ]:
voting_clf=VotingClassifier(estimators=[('lr',log),('rf',rnd),('svc',final_model)],voting='hard')
vote=voting_clf.fit(cv_train,Y_train)
y_pred=vote.predict(cv_test)
print("voting classifier", accuracy_score(Y_test, y_pred))

voting classifier 0.869
